In [1]:
import pandas as pd
import praw # Python Reddit API Wrapper

In [16]:
# Configuration
CLIENT_ID = 'b3CUxOmuvB9QeRasUu3Hew'
CLIENT_SECRET = 'CqqnRJucVB500qgPSWTH9wHHx2acQQ'
USER_AGENT = 'script:my_reddit_script:v1.0 (by /u/py_dev684)'

# Initialize the Reddit client
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

In [21]:

def fetch_top_comments(subreddits, post_limit=10, comment_limit=5):
    """Fetch top comments from specified subreddits """

    data = []  # List to store data before converting to DataFrame
    for subreddit_name in subreddits:
        print(f"Fetching from r/{subreddit_name}...")
        subreddit = reddit.subreddit(subreddit_name)
        # Fetch the top 10 hot posts
        for post in subreddit.hot(limit=post_limit):
            # Fetch and store the top 5 comments for the post
            post.comments.replace_more(limit=None)  # Replace "MoreComments" with actual comments
            for comment in post.comments[:comment_limit]:
                data.append({
                    'Subreddit': subreddit_name,
                    'Post Title': post.title,
                    'Comment Author': str(comment.author),  # Convert Redditor object to string
                    'Comment': comment.body
                })

    # Convert the data list into a DataFrame
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv('reddit_comments.csv', index=False)
    print("Data saved to reddit_comments.csv")

def fetch_comments_with_phrases(subreddits, phrases, comment_target=20):
    """Fetch comments that contain a target phrase, sourced from 'hot' posts from the specified subreddits. Comments are sourced from hot posts in each subreddit. The hot sorting method is a combination of factors that Reddit uses, including the age of the post and its upvote count. It's meant to show posts that are currently popular and relevant."""

    data = []  # List to store data before converting to DataFrame

    for subreddit_name in subreddits:
        print(f"Fetching from r/{subreddit_name}...")
        subreddit = reddit.subreddit(subreddit_name)
        
        for post in subreddit.hot(limit=None):  # Iterating through all hot posts
            if len(data) >= comment_target:  # If we've hit our comment target, stop.
                break

            post.comments.replace_more(limit=None)  # Replace "MoreComments" with actual comments
            
            # Filter comments based on presence of target phrases
            matching_comments = [comment for comment in post.comments if any(phrase.lower() in comment.body.lower() for phrase in phrases)]
            
            for comment in matching_comments:
                data.append({
                    'Subreddit': subreddit_name,
                    'Post Title': post.title,
                    'Comment Author': str(comment.author),
                    'Comment': comment.body,
                    'Upvotes': comment.score
                })

                if len(data) >= comment_target:  # If we've hit our comment target, stop.
                    break

    # Convert the data list into a DataFrame
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv('reddit_filtered_comments.csv', index=False)
    print("Data saved to reddit_filtered_comments.csv")

In [23]:
subreddits = ['finance']
target_phrases = ['bank run']

if __name__ == "__main__":
    # List of 5 subreddits
    subreddits = subreddits
    # List of target phrases
    phrases = target_phrases
    fetch_comments_with_phrases(subreddits, phrases, comment_target=2)

Fetching from r/finance...
Data saved to reddit_filtered_comments.csv
